In [ ]:
from datasets import load_from_disk, concatenate_datasets

# 加载多个数据集
dataset1 = load_from_disk("/mnt/file2/changye/NLPFINAL/casual_formal_sentence_pair_ACL170k_part1")
dataset2 = load_from_disk("/mnt/file2/changye/NLPFINAL/casual_formal_sentence_pair_ACL170k_part2")
dataset3 = load_from_disk("/mnt/file2/changye/NLPFINAL/casual_formal_sentence_pair_ACL170k_part3")
dataset4 = load_from_disk("/mnt/file2/changye/NLPFINAL/casual_formal_sentence_pair_ACL170k_part4")
dataset5 = load_from_disk("/mnt/file2/changye/NLPFINAL/casual_formal_sentence_pair_ACL170k_part5")
dataset6 = load_from_disk("/mnt/file2/changye/NLPFINAL/casual_formal_sentence_pair_ACL170k_part6")


# 合并数据集
combined_dataset = concatenate_datasets([dataset1, dataset2,dataset3,dataset4,dataset5,dataset6])

print(combined_dataset)


In [ ]:
train_test_split = combined_dataset.train_test_split(test_size=0.2, seed=42)

# 再将临时集拆分为验证集和测试集（各占 50%，即总数据的 10%）
val_test_split = train_test_split['test'].train_test_split(test_size=0.5, seed=42)

# 最终数据集
final_dataset = {
    'train': train_test_split['train'],
    'val': val_test_split['train'],
    'test': val_test_split['test']
}

# 打印拆分后的数据集信息
print("Train dataset size:", len(final_dataset['train']))
print("Validation dataset size:", len(final_dataset['val']))
print("Test dataset size:", len(final_dataset['test']))

In [ ]:
output_dir = "/mnt/file2/changye/dataset/casual_formal_sentence_pair_ACL170k"  # 设置保存目录
final_dataset['train'].save_to_disk(f"{output_dir}/train")
final_dataset['val'].save_to_disk(f"{output_dir}/val")
final_dataset['test'].save_to_disk(f"{output_dir}/test")


In [ ]:
final_dataset

In [ ]:
from datasets import load_from_disk
test_dataset=load_from_disk("/mnt/file2/changye/dataset/ACL_clear/test")

In [ ]:
print(test_dataset)

In [3]:
import hashlib
def generate_text_hash(text: str) -> str:
    """
    Generate a unique identifier for the given text using SHA-256.

    Args:
        text (str): Input text.

    Returns:
        str: Unique hash for the text.
    """
    hash_object = hashlib.sha256(text.encode('utf-8'))
    return hash_object.hexdigest()
test_list=[]
for item in test_dataset:
    for test in item['texts']:
        test_list.append({
            'directory': item['directory'],
            'filename': item['filename'],
            'text': test,
            'hash': generate_text_hash(item['directory']+item['filename']+test)
        })

In [ ]:
print((test_list[0].keys()))

In [41]:
import re

def clean_text_english(text):
    # 移除多余的换行符，将多行文本合并为单段落
    text = re.sub(r'\s*\n\s*', ' ', text)
    # 替换多个连续空格为单个空格
    text = re.sub(r'\s{2,}', ' ', text)
    # 确保标点符号和单词之间的空格规范化
    text = re.sub(r'\s*([.,!?;:])\s*', r'\1 ', text)
    # 修复句号后需要的空格
    text = re.sub(r'([.!?])([A-Za-z])', r'\1 \2', text)
    # 去除括号及其内容
    text = re.sub(r'\(.*?\)', '', text)  # 去除圆括号及其中的内容
    text = re.sub(r'（.*?）', '', text)  # 去除中文圆括号及其中的内容
    
    # 去除 URL（标准格式）
    text = re.sub(r'https?://\S+', '', text)
    # 去除 URL（带有空格的格式）
    text = re.sub(r'https?\s*:\s*//(?:\S+\s*)*', '', text)
    
    # 再次替换多个连续空格为单个空格，以处理移除 URL 后可能出现的多余空格
    text = re.sub(r'\s{2,}', ' ', text)
    # 去除首尾空格
    text = text.strip()
    return text

def is_valid_sentence(sentence):
    """
    判断一个句子是否有效： 
    - 句子不应仅包含数字或URL
    - 句子应包含至少一个字母字符
    """
    if not sentence:
        return False
    # 排除仅包含 URL 或数字的句子
    if re.match(r'^https?\s*:\s*//', sentence) or re.match(r'^\d+$', sentence):
        return False
    # 需要包含至少一个字母字符
    if not re.search(r'[a-zA-Z]', sentence):
        return False
    return True

def split_and_process_texts(text_list, max_length=200):
    updated_text_list = []

    for text in text_list:
        text['text'] = clean_text_english(text['text'])
        
        if len(text['text']) > max_length:
            # 按标点分割文本，并只选择有效的句子
            sentences = re.split(r'(?<=[.!?])\s+', text['text'])
            current_chunk = ""
            for sentence in sentences:
                sentence = sentence.strip()
                if is_valid_sentence(sentence):  # 仅添加有效的句子
                    # 如果当前段落加上句子长度超出限制，将当前段落加入结果，并重新开始
                    if len(current_chunk) + len(sentence) > max_length:
                        updated_text_list.append({
                            'directory': text['directory'],
                            'filename': text['filename'],
                            'text': current_chunk.strip(),
                            'hash': generate_text_hash(text['directory'] + text['filename'] + current_chunk.strip())
                        })
                        current_chunk = sentence
                    else:
                        current_chunk += " " + sentence

            # 添加最后的剩余部分
            if current_chunk:
                updated_text_list.append({
                    'directory': text['directory'],
                    'filename': text['filename'],
                    'text': current_chunk.strip(),
                    'hash': generate_text_hash(text['directory'] + text['filename'] + current_chunk.strip())
                })
        else:
            # 如果文本长度小于等于限制，直接加入结果
            updated_text_list.append({
                'directory': text['directory'],
                'filename': text['filename'],
                'text': text['text'].strip(),
                'hash': generate_text_hash(text['directory'] + text['filename'] + text['text'].strip())
            })

    return updated_text_list

# 筛选和拆分
filtered_list = split_and_process_texts(test_list, max_length=300)


In [ ]:
count=0
for text in test_list:
    if len(text['text'])>50 and len(text['text'])<400:
        count+=1
print(count)

In [ ]:
print(len(filtered_list))
formal_list=[]
for item in filtered_list:
    if len(item['text'])>300 or len(item['text'])<75:
        continue
    formal_list.append(item)
print(len(formal_list))

In [ ]:
print(formal_list[-1])

In [ ]:
import torch
from datasets import Dataset
import hashlib


# 将数据转换为 Hugging Face 数据集格式
dataset = Dataset.from_dict({
    'directory': [entry['directory'] for entry in formal_list],
    'filename': [entry['filename'] for entry in formal_list],
    'text': [entry['text'] for entry in formal_list],
    'hash': [entry['hash'] for entry in formal_list]
})

# 显示数据集
print("Original Hugging Face dataset:")
print(dataset)

# 保存数据集到磁盘（使用 Arrow 格式）
dataset.save_to_disk("clear_ACL_sentence170k")



In [6]:
from datasets import load_from_disk
new_data=load_from_disk("/mnt/file2/changye/NLPFINAL/AI_eval_result/Qwen2.5-1.5B-Instruct-finetune_sentence_part1")
print(new_data)

Dataset({
    features: ['evaluation', 'directory', 'filename', 'formal_text', 'casual_text', 'Model_formal_text'],
    num_rows: 300
})


In [7]:
print(new_data[0]['evaluation'])


```json
{
    "Style Transfer Strength": 0.2,
    "Content Preservation": 0.1,
    "Fluency": 0.3
}
```


In [8]:
pattern = r'\d\.\d'
import re
Style_Transfer_Strength=[]
Content_Preservation=[]
Fluency=[]
valid=0
for item in new_data:
    text=item['evaluation']
    matches = re.findall(pattern, text)
    if len(matches)==3:
        valid+=1
        Style_Transfer_Strength.append(float(matches[0]))
        Content_Preservation.append(float(matches[1]))
        Fluency.append(float(matches[2]))
    else:
        continue
print(valid)

300


In [9]:
import numpy as np
print(f"Style Transfer Strength: ",np.array(Style_Transfer_Strength).mean())
print(f"Content Preservation: ",np.array(Content_Preservation).mean())
print(f"Fluency: ",np.array(Fluency).mean())


Style Transfer Strength:  0.163
Content Preservation:  0.263
Fluency:  0.2886666666666666
